# Multiclass Classification Tutorial
This notebook demonstrates how to train and test a multi-class classifier.  The classification problem is to determine if a signal is one of 4 possible singals using a CNN-based classifier.

## Load Packages
Load packages that will be used throughout this tutorial.

In [ ]:
import sys
sys.path.append("..")
# General python packages
import os
import matplotlib.pyplot as plt

# metrics from sci-kit learn
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, precision_recall_fscore_support

# pytorch packages
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn

# custom tutorial packages
from rfml_ed_material.utils.data_utils import IQ_Dataset, IQ_data_gen, create_signal_jsons
from rfml_ed_material.models.cnn_model import CNN_RF
from rfml_ed_material.utils.train_utils import train_func, predict_func

## Data Generation Parameters
Data generation parameters for Py-waspgen.

In [ ]:
num_seq = 5000        # number of sequences per signal type
seq_len = 256         # length of each sequence
bandwidth = 0.5       # bandwidth
cent_freq = 0.0       # center frequency
start = 0             # signal start time
duration = seq_len    # signal duration
snr = 10              # signal to noise ratio

Define the signal types in the form of a list - BPSK and QPSK

In [ ]:
# list of 4 signals
signal_list = [{"format": "ask", "order": 16, "label": "16ASK"},
               {"format": "pam", "order": 16, "label": "16PAM"},
               {"format": "psk", "order": 16, "label": "16PSK"},
               {"format": "qam", "order": 16, "label": "16QAM"}]

Py-waspgen loads configuration information from json files.  Use the *create_signal_jsons* function to create the py-waspgen configuration files.

The following cell first checks if the "configs'' directory exists and, if it does not, it creates the directory for storing the json configuration files.

In [ ]:
if not os.path.isdir('configs'):
    os.mkdir('configs')

create_signal_jsons('configs',
                    signal_list,
                    observation_duration=seq_len,
                    cent_freq=[cent_freq, cent_freq],
                    bandwidth=[bandwidth, bandwidth],
                    start=[start, start],
                    duration=[seq_len, seq_len],
                    snr=[snr, snr])

## Generate Data
Use py-waspgen to generate data for training, validation, and testing a multiclass classifier.

Create the list of configuration files.

In [ ]:
signal_filenames = ['configs/' + signal["label"] + '.json' for signal in signal_list]

Generate data using wrapper function

In [ ]:
data, labels, label_dict = IQ_data_gen(signal_filenames, num_seq, seq_len)

## Prepare Data for Training

Pytorch uses a data set class for the data.  See the pytorch documentation for more details.

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
# create dataset class
rf_dataset = IQ_Dataset(data, labels, label_dict)

The data set should be split into training, validation, and testing sets.  The training set is used to train the model.  The validation set is used to track the model's performance during training and monitor for overfitting.  The test set is used for final evaluation of the model.  

The cell below uses the *random_split* pytorch function.

In [ ]:
splits = [0.8, 0.1, 0.1]  # proportion for train, validation, and test sets

rf_train, rf_val, rf_test = random_split(rf_dataset, splits)


Pytorch uses data loaders to batch the data.  The cell below sets the batch size parameter and creates dataloaders for the training, validation, and test sets.

In [ ]:
batch_size = 256           # batch size for dataloader

# create dataloader
train_dataloader = DataLoader(rf_train, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(rf_val, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(rf_test, batch_size=batch_size, shuffle=False)

## Train Pytorch Model

Establish the Pytorch model.

In [ ]:
model = CNN_RF(len(signal_list))

Set the learning rate for the training process

In [ ]:
learning_rate = 0.0001  # learning rate for optimizer

Define the loss function and setup the optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Training Parameters

In [ ]:
epochs = 200        # number of training epochs
print_every_n = 10  # print loss every n epochs

In [ ]:
model, train_loss, val_loss = train_func(model,
                                         optimizer,
                                         loss_fn,
                                         train_dataloader,
                                         val_dataloader,
                                         epochs,
                                         print_every_n)

## Evaluate Training

Plot training and validation loss. This should be done after training or monitored during training.  This confirms that the model learning has stablized and that the model is not overfit.  The latter can be seen because the validation loss is not increasing.

In [ ]:
fig, ax = plt.subplots()
ax.plot(train_loss, color='b', label="Train")
ax.plot(val_loss, color='r', label="Validation")
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.set_title('Training and Validation Loss')
fig.legend()

### Evaluation Metrics

Use the custom *predict_func* to extract the true targets from the test data loader and the predicted values for the test set from the learned model.

In [ ]:
y_test, y_pred = predict_func(model, test_dataloader)

Display the confusion matrix for the test set.

In [ ]:
labels = [signal["label"] for signal in signal_list]
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=labels, xticks_rotation='vertical')

Calculate the accuracy of the model on the test set.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy)

For multiclass probelms, metrics such as precision, recall, and f1 score must be calculated per class.  

In [ ]:
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred)
print('Precision for each class: ', precision)
print('Recall for each class: ', recall)
print('F1 score for each class: ', fscore)